In [ ]:
from numqt import *

In [ ]:
#-----------------------------------------------------------
# Characteristic lengths of the problem
#-----------------------------------------------------------
a0 = 1 # Bohr radius
Lx = 20 # Confinement length along x
Ly = 20 # Confinement length along y
Lz = 20 # Confinement length along z

#-----------------------------------------------------------
# Problem's parameters
#-----------------------------------------------------------
hbar = 1 # reduced planck constant
m = 1 # e-mass
uu = 1 # reduced mass (nucleus + electron)

#-----------------------------------------------------------
# Simulation parameters
#-----------------------------------------------------------
xbounds = (-Lx/2, Lx/2) # confinement length
ybounds = (-Ly/2, Ly/2) # confinement length
zbounds = (-Lz/2, Lz/2) # confinement length

dx_max = 0.3 # Global maximum allowed spacing
dy_max = 0.3 # Global maximum allowed spacing
dz_max = 0.3 # Global maximum allowed spacing


#-----------------------------------------------------------
# Creating grid
#-----------------------------------------------------------
mesh_obj = Mesh(dims=3,
                 xbounds=xbounds,
                 ybounds=ybounds,
                 zbounds=zbounds,
                 dx_max=dx_max,
                 dy_max=dy_max,
                 dz_max=dz_max,
                 dx_func = None,
                 dy_func= None,
                 dz_func= None,
                 max_iter= 10)

dimx = mesh_obj.Nx
dimy = mesh_obj.Ny
dimz = mesh_obj.Nz
mesh_obj.visualize_slice("x", 0, alpha = 0.5)
print(dimx, dimy, dimz)

In [ ]:
#-----------------------------------------------------------
# Obtaining canonical operators
#-----------------------------------------------------------
operators = canonic_ops(mesh_obj, additional_subspaces = None, hbar=1)
px2 = operators.get_ops()["p2"][0]
x2 = operators.get_ops()["x2"][0]

py2 = operators.get_ops()["p2"][1]
y2 = operators.get_ops()["x2"][1]

pz2 = operators.get_ops()["p2"][2]
z2 = operators.get_ops()["x2"][2]

r_inv = sparse.diags(1/(np.sqrt(x2.diagonal() + y2.diagonal() + z2.diagonal())+ np.ones(x2.shape[0])*1e-10), format = "csr")
#-----------------------------------------------------------
# Constructing the Hamiltonian
#-----------------------------------------------------------
H = Hamiltonian((px2 + py2 + pz2) / (2*uu) - r_inv /(uu*a0), mesh_obj)

In [ ]:
k = 10
energies, wavefunctions = H.solve(k)

In [ ]:
H.plot(0)

In [ ]:
# Define the analytical energy function
def energy(n):
    return -hbar**2 / (2 * m * a0**2  * n**2)

# Generate analytical energy levels
analytical_energies = sorted([energy(n) for n in range(1, k+1)])[:k]

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(range(1,k+1), analytical_energies, c="r", s=20, label="Analytics")
plt.scatter(range(1,k+1), energies, c="b", s=20, label="Numerics")
plt.legend()
plt.ylabel("Energy (E)")
plt.xlabel("Energy Level (n)")
plt.title("Comparison of Analytical and Numerical Energy Levels")
plt.grid(True)

plt.show()